In [22]:
import tkinter as tk
from PIL import Image, ImageDraw
import os

DIGIT = "9"  # CHANGE THIS FROM 0 TO 9
SAVE_DIR = f"data/{DIGIT}"

os.makedirs(SAVE_DIR, exist_ok=True)

count = 0
W, H = 280, 280

root = tk.Tk()
root.title(f"Draw digit {DIGIT}")

canvas = tk.Canvas(root, width=W, height=H, bg="black")
canvas.pack()

image = Image.new("L", (W, H), "black")
draw = ImageDraw.Draw(image)

def draw_lines(event):
    x, y = event.x, event.y
    r = 14
    canvas.create_oval(x-r, y-r, x+r, y+r, fill="white", outline="white")
    draw.ellipse([x-r, y-r, x+r, y+r], fill="white")

canvas.bind("<B1-Motion>", draw_lines)

def save():
    global count
    img = image.resize((28, 28))
    img.save(f"{SAVE_DIR}/{count}.png")
    count += 1
    clear()
    label.config(text=f"Saved: {count}")

def clear():
    canvas.delete("all")
    draw.rectangle([0, 0, W, H], fill="black")

tk.Button(root, text="Save", command=save).pack()
tk.Button(root, text="Clear", command=clear).pack()

label = tk.Label(root, text="Draw & Save", font=("Arial", 14))
label.pack()

root.mainloop()


In [24]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

layers = tf.keras.layers
models = tf.keras.models

X = []
y = []

for digit in range(10):
    folder = f"data/{digit}"
    for file in os.listdir(folder):
        img = Image.open(os.path.join(folder, file)).convert("L")
        img = np.array(img) / 255.0
        X.append(img)
        y.append(digit)

X = np.array(X).reshape(-1, 28, 28, 1)
y = np.array(y)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(X, y, epochs=20)
model.save("my_digit_model.h5")

print("✅ Model trained on YOUR handwriting")


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2397 - loss: 2.2090  
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7924 - loss: 1.4299 
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8760 - loss: 0.5637
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9065 - loss: 0.3522
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9138 - loss: 0.3410 
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9501 - loss: 0.1943 
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9717 - loss: 0.1500 
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9839 - loss: 0.0845
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9966 - loss: 0.0498
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9973 - loss: 0.0220
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0183
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy:

✅ Model trained on YOUR handwriting


In [26]:
import tkinter as tk
from PIL import Image, ImageDraw
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model("my_digit_model.h5")

W, H = 280, 280
root = tk.Tk()
root.title("Digit Recognizer")

canvas = tk.Canvas(root, width=W, height=H, bg="black")
canvas.pack()

image = Image.new("L", (W, H), "black")
draw = ImageDraw.Draw(image)

def draw_lines(event):
    x, y = event.x, event.y
    r = 14
    canvas.create_oval(x-r, y-r, x+r, y+r, fill="white", outline="white")
    draw.ellipse([x-r, y-r, x+r, y+r], fill="white")

canvas.bind("<B1-Motion>", draw_lines)

def predict():
    img = image.resize((28,28))
    img = np.array(img) / 255.0
    img = img.reshape(1,28,28,1)

    pred = model.predict(img)
    digit = np.argmax(pred)

    label.config(text=f"Prediction: {digit}")

def clear():
    canvas.delete("all")
    draw.rectangle([0,0,W,H], fill="black")

tk.Button(root, text="Predict", command=predict).pack()
tk.Button(root, text="Clear", command=clear).pack()

label = tk.Label(root, text="Draw a digit", font=("Arial", 16))
label.pack()

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
